In [13]:
import tensorflow as tf
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICE"]="1"
import scipy
import csv
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image
import SimpleITK as sitk

%matplotlib inline

In [31]:
def getROICenters(itk_image, radius):
    itk_image = sitk.BinaryMorphologicalClosing(itk_image, radius)
    itk_image = sitk.ConnectedComponent(itk_image,fullyConnected=True)
    label_shape_analysis = sitk.LabelShapeStatisticsImageFilter()
    nodules = []
    label_shape_analysis.Execute(itk_image)
    num_labels = label_shape_analysis.GetNumberOfLabels()
    backg=0
    label_shape_analysis.SetBackgroundValue(backg)
    print(num_labels,backg)
    for l_i in range(1, num_labels + 1):
        center = label_shape_analysis.GetCentroid(l_i)
        nodules.append(center)
        print("Center:", center)
    print("found n nodules", len(nodules))
    return nodules

In [ ]:
# vector index selection 
# make coordinate file for testresults 
testResultPath = '/home/fengkai/fengkai/U-net-prediction-images/predict/'

import pylab
from pylab import imshow
import csv 

testResult= os.listdir(testResultPath)


totalnumberofTest = len(testResult)

split=0.6
coordinateofTest=np.zeros((10000,3))
count =0

allcountbefore=0
for count in range(totalnumberofTest):

    fname = testResultPath+'{}.png'.format(count)
    print(fname)
    itk_image = sitk.ReadImage(fname)
    itk_image = sitk.Cast(itk_image, sitk.sitkUInt64)
    
    slice_sitk = sitk.GetArrayFromImage(itk_image)
    print(np.shape(slice_sitk))
    up = np.divide(np.max(slice_sitk),1.00000)
    slice_sitk =sitk.BinaryThreshold(slice_sitk,split,up,1,0)
    
    imshow(slice_sitk)
    nodules=getROICenters(itk_image,0)
    pylab.show()
    labels =len(nodules)
    la=0
    for nodule in nodules:
        coordinateofTest[allcountbefore+la,0]=number
        coordinateofTest[allcountbefore+la,1]=nodules[la][0]#x 
        coordinateofTest[allcountbefore+la,2]=nodules[la][1]#y
        la = la+1
    
    allcountbefore = allcountbefore+labels
    

with open('coordinateTest.txt', 'w') as f:

    for n,x, y in coordinateofTest: 
        if n !=0:
            f.write('{} {} {}\n'.format(n,x, y))
